# LightGBM

Script com foco no processo de forecast com previsão multi-step. <br>Método: direct prediction (ver https://machinelearningmastery.com/multi-step-time-series-forecasting/)


In [1]:
#https://www.youtube.com/watch?v=fG8H-0rb0mY
#https://machinelearningmastery.com/light-gradient-boosted-machine-lightgbm-ensemble/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import lightgbm as lgb
#lgb.__version__

In [2]:
plt.style.use('fivethirtyeight') # estilo dos gráficos
rcParams['figure.figsize'] = 15, 5 # tamanho das figuras

In [3]:
def load_data():
    """
    Função para ler e transformar os dados já presentes no diretório especificado
    """
    path = "../data/daily_load.csv"
    df_load = pd.read_csv(path, parse_dates = ["date"])
    df_load2 = df_load[df_load["id_reg"] == "S"]           # região sul
    df_load3 = df_load2[df_load2["date"] <= '2022-05-31']  # data de corte
    df_load4 = df_load3[["date", "load_mwmed"]].set_index("date")
    return df_load4

def train_test_split(data, n_test):
    """
    Função para partir or dados em treino e teste
    """
    if isinstance(data, pd.DataFrame):
        train, test = data.iloc[:-n_test, :], data.iloc[-n_test:, :]
    elif isinstance(data, np.ndarray):
        train, test = data[:-n_test, :], data[-n_test:, :]
    return train, test

# https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis = 1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace = True)
    return agg

def lightgbm_forecast(train, testX):
	# transform list into array
	train = np.asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = lgb.LGBMRegressor(objective='regression', n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        yhat = lightgbm_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        print('>expected = %.1f, predicted = %.1f' % (testy, yhat))
    # estimate prediction error
    mae = mean_absolute_error(test[:, -1], predictions)
    mape = mean_absolute_percentage_error(test[:, -1], predictions)
    rmse = np.sqrt(mean_squared_error(test[:, -1], predictions))    
    return mae, mape, rmse, test[:, -1], predictions

def get_measures(forecast, test):
    """
    Função para obter medidas de acurária a partir dos dados de projeção e teste
    """
    #forecast.reset_index(drop = True, inplace = True)
    #test.reset_index(drop = True, inplace = True)
    #errors = [(test.iloc[i] - forecast.iloc[i])**2 for i in range(len(test))]
    if isinstance(forecast, pd.Series) and isinstance(test, pd.Series):
        errors = [(test.iloc[i] - forecast.iloc[i])**2 for i in range(len(test))]
    # else:
    #     errors = [(test.iloc[i][0] - forecast.iloc[i])**2 for i in range(len(test))]
    mae = mean_absolute_error(test, forecast)
    mse = mean_squared_error(test, forecast)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(test, forecast)
    # smape
    a = np.reshape(test.values, (-1,))
    b = np.reshape(forecast.values, (-1,))
    smape = np.mean(100*2.0 * np.abs(a - b) / (np.abs(a) + np.abs(b))).item()
    # dicionário com as medidas de erro
    measures = { "erro": sum(errors),
                 "mae": mae,
                 "mse": mse,
                 "rmse": rmse,
                 "mape": mape,
                 "smape": smape
                }
    # arredondamento
    # for key, item in measures.items():
    #     measures[key] = round(measures[key], 2)
    return measures


In [95]:
df = load_data()
df.interpolate(method = "linear", inplace = True)
#df = df[df.index <= '2022-04-30']
values = df.values.tolist()
lag = 60 
outs = 10
data = series_to_supervised(values, n_in = lag, n_out = outs, dropnan=False)
data.tail(5)

,var1(t-60),var1(t-59),var1(t-58),var1(t-57),var1(t-56),var1(t-55),var1(t-54),var1(t-53),var1(t-52),var1(t-51),...,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9)
8183,12049.438875,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,12151.257583,...,12520.803833,10525.490875,9074.211250,11648.709583,12162.756792,NaN,NaN,NaN,NaN,NaN
8184,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,12151.257583,12512.483583,...,10525.490875,9074.211250,11648.709583,12162.756792,NaN,NaN,NaN,NaN,NaN,NaN
8185,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,12151.257583,12512.483583,12271.662917,...,9074.211250,11648.709583,12162.756792,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8186,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,12151.257583,12512.483583,12271.662917,10395.100000,...,11648.709583,12162.756792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8187,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,12151.257583,12512.483583,12271.662917,10395.100000,9137.307083,...,12162.756792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# DATA DA ÚLTIMA LINHA EM "data"
df[df.load_mwmed == data["var1(t)"].iloc[-1]]

,load_mwmed
date,
2022-05-31,12162.756792


In [32]:
train, test = train_test_split(data, 10)
train

,var1(t-60),var1(t-59),var1(t-58),var1(t-57),var1(t-56),var1(t-55),var1(t-54),var1(t-53),var1(t-52),var1(t-51),...,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4800.650000,4899.800000,6261.554167,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4899.800000,6261.554167,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6261.554167,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333,7425.491667
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333,7425.491667,7505.575000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333,7425.491667,7505.575000,7532.275000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8173,14198.562958,11296.964583,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,...,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708
8174,11296.964583,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,...,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833
8175,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,...,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833,10525.490875
8176,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,...,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833,10525.490875,9074.211250


In [33]:
# DATA DA ÚLTIMA LINHA EM "data"
df[df.load_mwmed == train["var1(t)"].iloc[-1]]

,load_mwmed
date,
2022-05-21,10256.970375


# Usando SKTIME para projetar

In [93]:
df_sk = df.asfreq('D')
df_sk.head()

,load_mwmed
date,
2000-01-01,4800.650000
2000-01-02,4899.800000
2000-01-03,6261.554167
2000-01-04,6733.741667
2000-01-05,6961.170833


In [102]:
n_test = 10
X, y = train_test_split(df_sk, n_test) # split partitions

In [103]:
#https://towardsdatascience.com/multi-step-time-series-forecasting-with-arima-lightgbm-and-prophet-cc9e3f95dfb0

from sktime.forecasting.compose import make_reduction
regressor = lgb.LGBMRegressor()
forecaster = make_reduction(regressor, window_length=60, strategy="recursive")
forecaster.fit(X)   # fit 
forecaster.predict(fh = [x for x in range(1,n_test + 1)])    # forecast three days ahead

c:\Users\user\anaconda3\lib\site-packages\sktime\utils\datetime.py:108: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if not hasattr(x, "freq") or x.freq is None:
c:\Users\user\anaconda3\lib\site-packages\sktime\utils\datetime.py:110: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  by *= x.freq
c:\Users\user\anaconda3\lib\site-packages\sktime\forecasting\base\_fh.py:565: FutureWarning: Timestamp.freqstr is deprecated and will be removed in a future version.
  cutoff = _coerce_to_period(cutoff, freq=cutoff.freqstr)


,load_mwmed
2022-05-22,8945.294801
2022-05-23,11716.222344
2022-05-24,12023.280010
2022-05-25,12047.558935
2022-05-26,11899.052396
2022-05-27,11931.996763
2022-05-28,10331.229395
2022-05-29,8786.639132
2022-05-30,11563.565119
2022-05-31,11801.086979


# MANUALMENTE

In [96]:
X, y = train_test_split(df, 10) # split partitions
train

,var1(t-60),var1(t-59),var1(t-58),var1(t-57),var1(t-56),var1(t-55),var1(t-54),var1(t-53),var1(t-52),var1(t-51),...,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4800.650000,4899.800000,6261.554167,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4899.800000,6261.554167,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6261.554167,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333,7425.491667
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6733.741667,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333,7425.491667,7505.575000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6961.170833,7110.362500,7105.354167,6307.487500,5523.620833,7111.320833,7435.058333,7425.491667,7505.575000,7532.275000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8173,14198.562958,11296.964583,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,...,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708
8174,11296.964583,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,...,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833
8175,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,...,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833,10525.490875
8176,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,...,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12520.803833,10525.490875,9074.211250


In [43]:
# EXEMPLO 1: t+3
# ÚLTIMO VALOR EM var1(t+h) DEVE SER O MESMO PARA TODAS ESTIMAÇÕES
# PARA VISUALIZAR MELHOR, OLHAR EXCEL "multistep" NA PASTA "DATA"
response_vars = data.columns[-outs:]
cols = [x for x in data.columns[:lag]]
cols.append("var1(t+3)")
data_ = train[cols]
#data_.dropna(inplace = True) # retirar "na" só da coluna de t+h que está se estimando 
data_.tail(5)

,var1(t-60),var1(t-59),var1(t-58),var1(t-57),var1(t-56),var1(t-55),var1(t-54),var1(t-53),var1(t-52),var1(t-51),...,var1(t-9),var1(t-8),var1(t-7),var1(t-6),var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1),var1(t+3)
8173,14198.562958,11296.964583,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,...,7742.394500,10808.548667,11654.623081,11608.524663,11361.001881,11305.707062,10060.549787,8994.856748,11424.087542,11802.526458
8174,11296.964583,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,...,10808.548667,11654.623081,11608.524663,11361.001881,11305.707062,10060.549787,8994.856748,11424.087542,11964.909375,10256.970375
8175,9206.600458,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,...,11654.623081,11608.524663,11361.001881,11305.707062,10060.549787,8994.856748,11424.087542,11964.909375,12269.051375,8938.579125
8176,11830.310375,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,...,11608.524663,11361.001881,11305.707062,10060.549787,8994.856748,11424.087542,11964.909375,12269.051375,12021.415458,11713.104333
8177,12475.772542,12928.446208,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,11890.737708,...,11361.001881,11305.707062,10060.549787,8994.856748,11424.087542,11964.909375,12269.051375,12021.415458,11802.526458,12054.195042


In [34]:
# EXEMPLO 2: t+4- PARA VISUALIZAR MELHOR, OLHAR EXCEL "multistep" NA PASTA "DATA"
# ÚLTIMO VALOR EM var1(t+h) DEVE SER O MESMO PARA TODAS ESTIMAÇÕES
# PARA VISUALIZAR MELHOR, OLHAR EXCEL "multistep" NA PASTA "DATA"
response_vars = data.columns[-outs:]
cols = [x for x in data.columns[:lag]]
cols.append("var1(t+4)")
data_ = data[cols]
data_.dropna(inplace = True) # retirar "na" só da coluna de t+h que está se estimando 
data_.tail(5)

C:\Users\user\AppData\Local\Temp\ipykernel_8884\2497736831.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_.dropna(inplace = True) # retirar "na" só da coluna de t+h que está se estimando


,var1(t-60),var1(t-59),var1(t-58),var1(t-57),var1(t-56),var1(t-55),var1(t-54),var1(t-53),var1(t-52),var1(t-51),...,var1(t-9),var1(t-8),var1(t-7),var1(t-6),var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1),var1(t+4)
8179,12481.912583,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,...,10060.549787,8994.856748,11424.087542,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,12520.803833
8180,11564.961500,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,...,8994.856748,11424.087542,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,10525.490875
8181,9796.231333,8644.748083,12049.438875,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,...,11424.087542,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,9074.211250
8182,8644.748083,12049.438875,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,...,11964.909375,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,11648.709583
8183,12049.438875,12864.373792,12801.830000,11890.737708,11762.593583,10305.348750,9318.594333,12069.142875,11989.101125,12151.257583,...,12269.051375,12021.415458,11802.526458,10256.970375,8938.579125,11713.104333,12054.195042,12186.721375,12482.523708,12162.756792


In [39]:
# DIRECT PREDICTION V1: ESTÁ CONSIDERANDO OS DADOS DE TESTE T + H, QUE NÃO ESTARÃO DISPONÍVEIS PARA FORECAST

# lista com as variáveis resposta
response_vars = data.columns[-outs:]

predictions = list()
# estimate models for every t + h
for response in response_vars:
    cols = [x for x in data.columns[:lag]] # features names. resets every iteration
    cols.append(response)
    data_ = data[cols]
    X, y = data_.iloc[:, :-1], data_.iloc[:, -1]    
    model = lgb.LGBMRegressor(objective='regression', n_estimators=1000)
    model.fit(X, y)
    yhat = model.predict([data_.iloc[-1, :-1].values])[0]
    yobs = y.iloc[-1] 
    print(f"> expected: {yobs}, predicted: {yhat}")
    predictions.append(yhat)

> expected: 12162.75679167, predicted: 12125.3062097979
> expected: nan, predicted: 680.4406038926243
> expected: nan, predicted: 411.9791824649825
> expected: nan, predicted: 493.16170290447366


KeyboardInterrupt: 

In [35]:
# DIRECT PREDICTION V2. Problema: valor observado está vazio para t + h para h > 0 em 30/04/2022 em diante

# lista com as variáveis resposta
response_vars = data.columns[-outs:]

nrows = data.shape[0]

predictions = list()
# estimate models for every t + h
nrows = data.shape[0]
i = 0
for response in response_vars:
    cols = [x for x in data.columns[:lag]] # features names. resets every iteration
    cols.append(response)
    data_ = data[cols]
    nrows -= i
    #print(nrows)
    X, y = data_.iloc[:nrows, :-1], data_.iloc[:nrows, -1]  
    #print(y.shape)  
    #print(y)
    model = lgb.LGBMRegressor(objective='regression', n_estimators=1000)
    model.fit(X, y)
    yhat = model.predict([data_.iloc[-1, :-1].values])[0]
    yobs = data["var1(t)"].iloc[i - 1] # !!!!!!!!    VER AQUI: t + h = i - 1 ... (load_mwmed de 30/04/2022 em diante) !!!!!!!!!!!!! 
    print(f"> expected: {yobs}, predicted: {yhat}")
    predictions.append(yhat)
    i += 1

> expected: 12162.75679167, predicted: 12125.3062097979
> expected: 4800.65, predicted: 11737.872327048399
> expected: 4899.8, predicted: 12095.795487566218
> expected: 6261.55416667, predicted: 11699.701333758983
> expected: 6733.74166667, predicted: 9936.527196361329
> expected: 6961.17083333, predicted: 8209.772969299112
> expected: 7110.3625, predicted: 11508.61695530888
> expected: 7105.35416667, predicted: 11727.951454302902
> expected: 6307.4875, predicted: 12085.249109471068
> expected: 5523.62083333, predicted: 12083.243791807303


In [ ]:
# DIRECT PREDICTION V2. Problema: valor observado está vazio para t + h para h > 0 em 30/04/2022 em diante

# lista com as variáveis resposta
response_vars = data.columns[-outs:]

nrows = data.shape[0]

predictions = list()
# estimate models for every t + h
nrows = data.shape[0]
i = 0
for response in response_vars:
    cols = [x for x in data.columns[:lag]] # features names. resets every iteration
    cols.append(response)
    data_ = data[cols]
    nrows -= i
    #print(nrows)
    X, y = data_.iloc[:nrows, :-1], data_.iloc[:nrows, -1]  
    #print(y.shape)  
    #print(y)
    model = lgb.LGBMRegressor(objective='regression', n_estimators=1000)
    model.fit(X, y)
    yhat = model.predict([data_.iloc[-1, :-1].values])[0]
    yobs = data["var1(t)"].iloc[i - 1] # !!!!!!!!    VER AQUI: t + h = i - 1 ... (load_mwmed de 30/04/2022 em diante) !!!!!!!!!!!!! 
    print(f"> expected: {yobs}, predicted: {yhat}")
    predictions.append(yhat)
    i += 1